# Chapter 8 Trees

A popular method to price derivatives is to use trees. Each node in a tree is splitted in different outcomes, so every time the tree advances (creating new nodes) new situations are generated from the previous node.

Imagine a binary tree in a Brownian motion. Every time the node is splitted there is the situation where the Spot increases and anoter situation where the stock decreases (with a variation according to the time lapse between nodes and the volatility).

Here the basics of the trees will be presented.

### The TreeProduct class

We start with the base class.

In [12]:
from math import exp, sqrt, log
import numpy as np

In [2]:
class TreeProduct:
    def __init__(self, FinalTime_):
        self._FinalTime = FinalTime_
        
    def FinalPayOff(self,Spot):
        pass
    
    def PreFinalValue(self, Spot, Time, DiscountedFutureValue):
        pass
    
    def __del__(self):
        del self
        
    def clone(self):
        from copy import deepcopy
        return deepcopy(self)
    
    def GetFinalTime(self):
        return self._FinalTime         

### Inhereted classes

Each class works for a different type of option (american and european).

In [140]:
class TreeAmerican(TreeProduct):
    def __init__(self,FinalTime_, ThePayOff_):
        super().__init__(FinalTime_)
        self.__ThePayOff = ThePayOff_
        
    def FinalPayOff(self, Spot):
        return self.__ThePayOff(Spot)
    
    def PreFinalValue(self, Spot, ThisTime, DiscountedFutureValue):
        return max(self.__ThePayOff(Spot), DiscountedFutureValue)        

In [139]:
class TreeEuropean(TreeProduct):
    def __init__(self,FinalTime_, ThePayOff_):
        super().__init__(FinalTime_)
        self.__ThePayOff = ThePayOff_
        
    def FinalPayOff(self, Spot):
        return self.__ThePayOff(Spot)
    
    def PreFinalValue(self, Spot, ThisTime, DiscountedFutureValue):
        return DiscountedFutureValue

### The binomial tree class

The book uses the class pair to store the values of the spots and the option values. In python we can do the following.

In [6]:
class pair:
    def __init__(self):
        self.first = 0
        self.second = 0

Or better

In [80]:
from dataclasses import dataclass
@dataclass
class pair:
    first: float
    second: float

Now we will create the class that compues everything in the creation of the tree.

In [143]:
class SimpleBinomialTree:
    def __init__(self, Spot_, r_, d_, Volatility_, Steps, Time):
        self.__Spot = Spot_
        self.__r = r_
        self.__d = d_
        self.__Volatility = Volatility_
        self.__Steps = Steps
        self.__Time = Time
        self.__TreeBuilt = False
        self.__TheTree = []
        self.__Discounts = [0 for i in range(self.__Steps)]
    
    
    def __BuildTree(self):
        self.__TreeBuilt = True
        self.__TheTree = [[] for i in range(self.__Steps+1)]
        
        InitialLogSpot = log(Spot)
        
        for i in range(self.__Steps+1):
            self.__TheTree[i] = [pair(0,0) for i in range(i+1)]
            thisTime = i*self.__Time/self.__Steps
            
            movedLogSpot = InitialLogSpot + self.__r.Integral(0, thisTime) - self.__d.Integral(0, thisTime)
            
            movedLogSpot -= 0.5*self.__Volatility*self.__Volatility*thisTime
            
            sd = self.__Volatility*sqrt(self.__Time/self.__Steps)
                        
            k = 0
            for j in range(-i,i+2,2):
                self.__TheTree[i][k].first = exp(movedLogSpot + j*sd)
                
                k+=1
            
            for l in range(self.__Steps):
                self.__Discounts[l] = exp(-self.__r.Integral(l*self.__Time/self.__Steps,(l+1)*self.__Time/self.__Steps))
    
    def GetThePrice(self, TheProduct):
        if (not self.__TreeBuilt):
            self.__BuildTree()
            
        if (TheProduct.GetFinalTime() != self.__Time):
            raise ValueError('Mismatched product in SimpleBinomialTree')
            
        k = 0
        for j in range(-self.__Steps,self.__Steps+2,2):
            
            self.__TheTree[self.__Steps][k].second = TheProduct.FinalPayOff(
                self.__TheTree[self.__Steps][k].first)
            k+=1
            
        for i in range(1,self.__Steps+1,1):
            index = self.__Steps - i
            ThisTime = index*self.__Time / self.__Steps
            
            k = 0
            for j in range(-index,index+2,2):
                Spot = self.__TheTree[index][k].first
                futureDiscountedValue = 0.5*self.__Discounts[index]*(self.__TheTree[index+1][k].second + 
                                                              self.__TheTree[index+1][k+1].second)
            
                self.__TheTree[index][k].second = TheProduct.PreFinalValue(Spot, ThisTime, futureDiscountedValue)
                k+=1
            
        return self.__TheTree[0][0].second


Let's put it all together.

In [16]:
class PayOff():
    def __init__(self):
        pass
    
    def __call__(self,Spot):
        pass    

class PayOffBridge():        
    def __del__(self):
        del self
        
    def copy(self,InnerPayOff):
        from copy import deepcopy
        self = deepcopy(InnerPayOff)
        
class PayOffCall(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)    

In [17]:
class ParametersInner:
    def __init__(self):
        pass
    
    def Integral(self,time1, time2):
        pass
    
    def IntegralSquare(self,time1,time2):
        pass

class Parameters:    
    def RootMeanSquare(self,time1,time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def Mean(self,time1, time2):
        total = self.Integral(time1,time2)
        return total/(time2-time1)
    
    def copy(self,original):
        from copy import deepcopy
        self = deepcopy(InnerPayOff) 
        
    def __del__(self):
        del self    
        
class ParametersConstant(ParametersInner,Parameters):
    def __init__(self,constant):
        self.__Constant = constant
        self.__ConstantSquare = self.__Constant*self.__Constant
    
    def Integral(self, time1, time2):
        return (time2 - time1)*self.__Constant
    
    def IntegralSquare(self, time1, time2):
        return (time2 - time1)*self.__ConstantSquare   

We will use the same example we always use to price calls.

In [162]:
Expiry = 0.5
Strike = 40
Spot = 42
Vol = 0.2
r = 0.1
d = 0
Steps = 4000

In [163]:
thePayOff = PayOffCall(Strike)
rParam = ParametersConstant(r)
dParam = ParametersConstant(d)

europeanOption = TreeEuropean(Expiry, thePayOff)
americanOption = TreeAmerican(Expiry, thePayOff)

theTree = SimpleBinomialTree(Spot, rParam, dParam, Vol, Steps, Expiry)

europrice = theTree.GetThePrice(europeanOption)
americanprice = theTree.GetThePrice(americanOption)

print(f'Euro price {europrice}, American price {americanprice}')

Euro price 4.759410413326544, American price 4.759410413326544


The prices are close to the correct solution. Now we will try adding the dividend effect.

In [191]:
Expiry = 0.5
Strike = 40
Spot = 42
Vol = 0.2
r = 0.1
d = 0.1
Steps = 1000

In [165]:
thePayOff = PayOffCall(Strike)
rParam = ParametersConstant(r)
dParam = ParametersConstant(d)

europeanOption = TreeEuropean(Expiry, thePayOff)
americanOption = TreeAmerican(Expiry, thePayOff)

theTree = SimpleBinomialTree(Spot, rParam, dParam, Vol, Steps, Expiry)

europrice = theTree.GetThePrice(europeanOption)
americanprice = theTree.GetThePrice(americanOption)

print(f'Euro price {europrice}, American price {americanprice}')

Euro price 3.2794621977688543, American price 3.3272641439441206


### Forwards

The book also adds the forwards to illustrate their behaviour and compare it to the other options.

In [187]:
class PayOffForward(PayOff,PayOffBridge):
    def __init__(self, Strike):
        self.__Strike = Strike
        
    def __call__(self, Spot):
        return Spot - Strike

In [193]:
Expiry = 0.5
Strike = 40
Spot = 42
Vol = 0.2
r = 0.1
d = 0.1
Steps = 1000

In [194]:
rParam = ParametersConstant(r)
dParam = ParametersConstant(d)

In [197]:
forwardPayOff = PayOffForward(Strike)
forward = TreeEuropean(Expiry, forwardPayOff)

theTree = SimpleBinomialTree(Spot, rParam, dParam, Vol, Steps, Expiry)

forwardPrice = theTree.GetThePrice(forward)
actualForwardPrice = exp(-r*Expiry)*(Spot*exp((r-d)*Expiry)-Strike)

print(f'Binomial Forward price {forwardPrice}, Actual Forward price {actualForwardPrice}')
print(f'Error {100*(actualForwardPrice - forwardPrice) /actualForwardPrice}')

Binomial Forward price 1.9024575172872937, Actual Forward price 1.902458849001428
Error 6.999962890127958e-05


So we can see that the price is very very close to the real one.

### Exercises

**Exercise 8.1** Find a class that does barrier options in the same TreeProduct class hierarchy. Try it out. How stable is the price? How might you improve stability?

**Exercise 8.2**